In [1]:
import numpy as np
import hmvec as hm
import matplotlib.pyplot as plt
import matplotlib

In [2]:
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')

In [ ]:
from cobaya.run import run
from cobaya.theory import Theory
from cobaya.likelihood import Likelihood

In [3]:
#Plot settings
%matplotlib inline
matplotlib.rcParams['axes.labelsize'] = 'x-large'
matplotlib.rcParams['xtick.labelsize'] = 'x-large'
matplotlib.rcParams['ytick.labelsize'] = 'x-large'
matplotlib.rcParams['legend.fontsize'] = 'x-large'
matplotlib.rcParams['axes.titlesize'] = 'xx-large'
matplotlib.rcParams['figure.titlesize'] = 'xx-large'

## Setting Up

First, we setup our grid and initialize our halo model. Since this on a shared node on Cori, the (z,m,k) grid we set up is fairly coarse.

In [ ]:
#Setup Grid
Nz = 100                                 # num of redshifts
Nm = 100                                 # num of masses
Nk = 1000                                # num of wavenumbers
redshifts = np.linspace(0.01, 6, Nz)             
masses = np.geomspace(1.0e6, 1.0e15, Nm)          
ks = np.geomspace(1.0e-3, 100.0, Nk)              # wavenumbers
ells = np.linspace(10, 3000, 200)

In [ ]:
#Initialize Halo Model 
hcos = hm.HaloModel(redshifts, ks, ms=masses, mass_function='tinker')

In [ ]:
#params={'As':2.4667392631170437e-09,'ns':.96,'omch2':(0.25-.043)*.7**2,'ombh2':0.044*.7**2,'H0':70.},mdef='mean'

Next, set up the parameters for the CIB model we want. We'll be using Planck 2013 values. We can also give different parameter values (e.g. a dictionary of new values for some/all of the parameters); see 'set_cibParams' documentation for details.

In [ ]:
#Set CIB Parameters
hcos.set_cibParams('planck13')

## Frequencies

We need to create a 2x2 array of frequencies. The first axis contains the frequencies to be auto/cross-correlated. The second axis contains the endpoints of a range of frequencies (as in a bandpass).

If you can provide a 1D array containing a single frequency/bandpass, it will assume you want an autocorrelation.

In [ ]:
#Autocorrelation: 1 Freq
autofreq = np.array([545e9], dtype=np.double)          

#Autocorrelation: Bandpass
autoband = np.array([540e9, 550e9], dtype=np.double)          

#Cross-Correlation: 1 Freq
crossfreq = np.array([[545e9],[353e9]], dtype=np.double)  

#Cross-Correlation: Bandpass
crossband = np.array([[540e9, 550e9],[347e9, 359e9]], dtype=np.double)   

Let's just stick with a single frequency autocorrelation for now.

## Calculations

Let's do all of the calculations with both the Tinker and Jiang subhalo mass functions. The default in this notebook will be the Jiang (note that this is not the default value for the power functions, which choose Tinker by default).

In [ ]:
#Get 3D Power Spectra P(z,k): Tinker
Pjj_tot_tink = hcos.get_power("cib", "cib", nu_obs=autofreq, satmf = 'tinker')  
Pjj_1h_tink = hcos.get_power_1halo("cib", "cib", nu_obs=autofreq, satmf = 'tinker')  
Pjj_2h_tink = hcos.get_power_2halo("cib", "cib", nu_obs=autofreq, satmf = 'tinker')  
Pjj_cen = hcos.get_power("cib", "cib", nu_obs=autofreq, subhalos=False)  # no satellites

In [ ]:
#Limber Integrals: Tinker
C_tot_tink, dcdz_tot_tink = hcos.C_ii(ells, redshifts, ks, Pjj_tot_tink, dcdzflag=True)
C_1h_tink, dcdz_1h_tink = hcos.C_ii(ells, redshifts, ks, Pjj_1h_tink, dcdzflag=True)
C_2h_tink, dcdz_2h_tink = hcos.C_ii(ells, redshifts, ks, Pjj_2h_tink, dcdzflag=True)
C_cen, dcdz_cen = hcos.C_ii(ells, redshifts, ks, Pjj_cen, dcdzflag=True)

In [ ]:
#Get 3D Power Spectra P(z,k): Jiang
Pjj_tot = hcos.get_power("cib", "cib", nu_obs=autofreq, satmf = 'jiang')  
Pjj_1h = hcos.get_power_1halo("cib", "cib", nu_obs=autofreq, satmf = 'jiang')  
Pjj_2h = hcos.get_power_2halo("cib", "cib", nu_obs=autofreq, satmf = 'jiang')  
Pjj_cen = hcos.get_power("cib", "cib", nu_obs=autofreq, subhalos=False, satmf = 'jiang')  # no satellites

In [ ]:
#Limber Integrals: Jiang
C_tot, dcdz_tot = hcos.C_ii(ells, redshifts, ks, Pjj_tot, dcdzflag=True)
C_1h, dcdz_1h = hcos.C_ii(ells, redshifts, ks, Pjj_1h, dcdzflag=True)
C_2h, dcdz_2h = hcos.C_ii(ells, redshifts, ks, Pjj_2h, dcdzflag=True)
C_cen, dcdz_cen = hcos.C_ii(ells, redshifts, ks, Pjj_cen, dcdzflag=True)

## Toy Data for MCMC Code

In [ ]:
xdata = ells[1::int(len(ells)/20)]
xdata

In [ ]:
model = np.stack((ells, C_tot), axis=-1)
np.save('toy_model', model)

In [ ]:
sigma = 5.0e2

Use the Lloyd Knox formula for the error estimates.

In [ ]:
ydata = np.random.normal(loc=C_tot[1::int(len(ells)/20)]* 3e-15, scale=sigma) 

In [ ]:
plt.errorbar(xdata,ydata, yerr=sigma, fmt='.')
plt.plot(ells, C_tot * 3e-15);

In [ ]:
np.sqrt(np.sum((ydata/sigma)**2))

In [ ]:
cov = np.identity(len(ydata)) * sigma

In [ ]:
np.save('toy_cov', cov)

In [ ]:
np.save('toy_data', np.stack((xdata, ydata), axis=-1))

## Cobaya

In [ ]:
#Toy Data
filename_cov = 'toy_cov.npy'
filename_data = 'toy_data.npy'
cov = np.load(filename_cov)
data = np.load(filename_data)

#Autocorrelation: 1 Freq
autofreq = np.array([545e9], dtype=np.double)    

#Cobaya Input File
info = {
    "likelihood": {"src_class.ChiSqLikelihood": {"python_path": "~/git/hmvec/"}} ,
    
    "params": dict([
        #CIB Model Parameters
        ("alpha", {
            "prior": {"min": 0, "max": 1.3},
            "ref": {"min": 0.2, "max": 0.5},
            "latex": r"\alpha"}),
        ("beta", {
            "prior": {"min": 0, "max": 2.1},
            "ref": {"min": 1.2, "max": 1.7},
            "latex": r"\beta"}),
        ("gamma", {
            "prior": {"min": 0, "max": 2.7},
            "ref": {"min": 1.2, "max": 1.7},
            "latex": r"\gamma"}),
        ("delta", {
            "prior": {"min": 2.5, "max": 4.6},
            "ref": {"min": 3, "max": 4},
            "latex": r"\delta"}),
        ("Td_o", {
            "prior": {"min": 15, "max": 30},
            "ref": {"min": 18, "max": 22},
            "latex": r"T_{d,o}"}),
        ("logM_eff", {
            "prior": {"min": 11, "max": 14},
            "ref": {"min": 11.8, "max": 13},
            "latex": r"\text{log}(M_{\text{eff}})"}),
        ("L_o", {
            "prior": {"min": 1e-17, "max": 1e-13},
            "ref": {"min": 9e-16, "max": 9e-15},
            "latex": r"L_o"}),
        
        #Fixed Params for Theory/Likelihood
        ("data", data), 
        ("covariance", cov),
        ("freq", autofreq)
    ]),

    "sampler": {
        "mcmc": {"Rminus1_stop": 0.001, "max_tries": 1000}
    },

    "output": "toy/autocib"
}

#Run Cobaya
updated_info, sampler = run(info)

## HaloGen Code

## Plots

Let's first look at the $C$'s with the centrals and the satellites.

In [ ]:
nfn = np.load('nfn.npy')
# plt.plot(masses, nfn[:,0])
nfn.shape

In [ ]:
plt.figure(figsize=(10,7))

#Plot C's
plt.loglog(ells, C_tot, label='total')
plt.loglog(ells, C_1h, label='1 halo term')
plt.loglog(ells, C_2h, label='2 halo term')

#Gravy
plt.xlabel(r'$\ell$')
plt.ylabel(rf'$C^{{ {autofreq[0]/1e9:.0f} \; x \; {autofreq[0]/1e9:.0f} }}_\ell$')
plt.legend();

# plt.savefig('cii_1h2h_jiang.pdf', dpi=900, bbox_inches='tight');

Now let's see the total $C$ without any satellites. Note the difference in the magnitude of the power with and without the satellites. The units are not established yet.

In [ ]:
plt.figure(figsize=(10, 7))

#Plot C without Satellites
plt.loglog(ells, C_cen)

#Gravy
plt.xlabel(r'$\ell$')
plt.ylabel(rf'$C^{{ {autofreq[0]/1e9:.0f} \; x \; {autofreq[0]/1e9:.0f} }}_\ell$');

Now let's look at $dC/dz$ with the satellites.

In [ ]:
#Plot dC/dz With Satellites
test_ells = np.array([100, 300, 450, 500, 1000])
plt.figure(figsize=(10,7))
for ell in test_ells:
    #Get index
    i = np.where(abs(ell - ells) <= 1)[0][0]

    #Spectra
    plt.semilogy(redshifts, dcdz_tot[:, i], label=rf"$\ell = {ells[i]:0.0f}$")
    
    print(f'L = {ell}  :  {np.sum(dcdz_tot[:5, i])/np.sum(dcdz_tot[:, i]) * 100:.3f}%')

    #Gravy
    plt.xlabel(r'$z$')
    plt.ylabel(rf'$dC^{{ {autofreq[0]/1e9:.0f} \;x\; {autofreq[0]/1e9:.0f} }}_\ell / dz$')
    plt.legend()
    plt.minorticks_on()
    
plt.savefig('dcdz_545.pdf', bbox_inches='tight', dpi=600)

In [ ]:
u = hcos.uk_profiles['nfw']
plt.plot(ks, u[1, 135, :])
plt.xlabel('k')
plt.ylabel('NFW');

ux = np.fft.irfft(u)

In [ ]:
redshifts[:10]

In [ ]:
f'{masses[135]:.2e}'

In [ ]:
ells[:10]

In [ ]:
#Setup
plt.figure(figsize=(10,10))

#Plots
zlow = 0
ellow = 1
plt.contourf(ells[ellow:50], redshifts[zlow:100], dcdz_tot[zlow:100, ellow:50])

#Gravy
plt.colorbar()
plt.xlabel(r'$\ell$')
plt.ylabel('z')
plt.title(r'Total $dC$ (with satellites)')
# plt.xlim(10,750)

#Ticks
ax = plt.gca()
# ax.set_yticks(np.arange(0,7))
plt.minorticks_on()

# plt.savefig('dc_jiang.pdf', dpi=900, bbox_inches='tight');

In [ ]:
#Setup
plt.figure(figsize=(10,10))

#Plots
plt.contourf(ells, redshifts, dcdz_1h)

#Gravy
plt.colorbar()
plt.xlabel(r'$\ell$')
plt.ylabel('z')
plt.title(r'1h $dC$ (with satellites)')

#Ticks
ax = plt.gca()
ax.set_yticks(np.arange(0,7))
plt.minorticks_on()

# plt.savefig('dc_1h_jiang.pdf', dpi=900, bbox_inches='tight');

In [ ]:
#Setup
plt.figure(figsize=(10,10))

#Plots
plt.contourf(ells, redshifts, dcdz_2h)

#Gravy
plt.colorbar()
plt.xlabel(r'$\ell$')
plt.ylabel('z')
plt.title(r'2h $dC$ (with satellites)')

#Ticks
ax = plt.gca()
ax.set_yticks(np.arange(0,7))
plt.minorticks_on();

# plt.savefig('dc_2h_jiang.pdf', dpi=900, bbox_inches='tight');

And $dC/dz$ without the satellites.

In [ ]:
#Plot dC/dz With Satellites
test_ells = np.array([100, 300, 500, 1000])
plt.figure(figsize=(10,7))
for ell in test_ells:
    #Get index
    i = np.where(abs(ell - ells) <= 1)[0][0]

    #Spectra
    plt.semilogy(redshifts, dcdz_cen[:, i], label=rf"$\ell = {ells[i]:0.0f}$")

    #Gravy
    plt.xlabel(r'$z$')
    plt.ylabel(rf'$dC^{{ {autofreq[0]:.2e} \;x\; {autofreq[0]:.2e} }} / dz$')
    plt.legend()
    plt.minorticks_on();